In [4]:
from regelum.system import System
from regelum.simulator import SciPy
from regelum.simulator import CasADi
import numpy as np


class KinematicPoint(System):
    _name = "kinematic-point"
    _system_type = "diff_eqn"
    _dim_state = 2
    _dim_inputs = 2
    _dim_observation = 2
    _observation_naming = _state_naming = ["x", "y"]
    _inputs_naming = ["v_x", "v_y"]
    _action_bounds = [[-10.0, 10.0], [-10.0, 10.0]]

    def _compute_state_dynamics(self, time, state, inputs):
        return inputs  # The velocity inputs directly define the rate of change of position.

def get_action(state):
    return (
        -state
    )  # Stabilizing action: control input is the negative of the system state.

# Define the initial state (initial position of the kinematic point).
initial_state = np.array([[2.0, 2.0]])  # Start at position (2, 2)

# Initialize the kinematic point system.
kinematic_point = KinematicPoint()

# Instantiate a simulator for the kinematic point system.
simulator =  CasADi(  #SciPy(
    system=kinematic_point, state_init=initial_state, time_final=4, max_step=0.1
)

In [1]:
import pytest
import numpy as np

from regelum.scenario import Scenario
from regelum.simulator import SciPy
from regelum.simulator import CasADi

from src.system import HydraulicSystem
from src.policy import PDController

from regelum import callback

sampling_time = 1e-3

system = callback.detach(HydraulicSystem)() # HydraulicSystem()

simulator = CasADi( #SciPy(
    system=system,
    state_init = np.array([1e3, 0]),
    time_final = 10e-3,
)

scenario = Scenario(
    policy=PDController(
        system=system,
        sampling_time=sampling_time
    ),
    simulator=simulator,
    sampling_time=0.01,
    N_episodes=1,
    N_iterations=1,
)

scenario.run()

TypeError: RCTypeHandler.max() got multiple values for argument 'rc_type'

In [1]:
import numpy as np
from src.system import HydraulicSystem
from src.policy import PDController

from regelum.scenario import Scenario
from regelum.simulator import SciPy

# from regelum.system import System
from regelum.utils import rg


system = HydraulicSystem()

simulator = SciPy(
    system=system,
    state_init = np.array([1e3, 0, 0, 0]),
    time_final = 10e-3,
)

AttributeError: 'HydraulicSystem' object has no attribute 'compute_closed_loop_rhs'

In [2]:
system.compute_state_dynamics(0, np.array([2.0, 0, 0]), inputs=np.array([[10]]))

array([[    0.,     0., 10000.]])